In [12]:
import ccxt
import pandas as pd
from datetime import datetime
# pip install finance-datareader 패키지 이름 주의
import FinanceDataReader as fdr

In [9]:
def scrape_ohlcv(exchange, max_retries, symbol, timeframe, since, limit):
    earliest_timestamp = exchange.milliseconds() # 현재시간?
    timeframe_duration_in_seconds = exchange.parse_timeframe(timeframe) # 초단위로 바꿈
    timeframe_duration_in_ms = timeframe_duration_in_seconds * 1000
    timedelta = limit * timeframe_duration_in_ms
    if isinstance(since, str):
        since = exchange.parse8601(since)

    all_ohlcv = []
    while True:
        fetch_since = earliest_timestamp - timedelta
        ohlcv = retry_fetch_ohlcv(exchange, max_retries, symbol, timeframe, fetch_since, limit)
        # if we have reached the beginning of history
        if ohlcv[0][0] >= earliest_timestamp:
            break
        earliest_timestamp = ohlcv[0][0]
        all_ohlcv = ohlcv + all_ohlcv
        if len(all_ohlcv) % 100000 == 0:
            print(len(all_ohlcv), symbol, 'candles in total from', exchange.iso8601(all_ohlcv[0][0]), 'to', exchange.iso8601(all_ohlcv[-1][0]))
        # if we have reached the checkpoint
        if fetch_since < since:
            break
    return all_ohlcv

def retry_fetch_ohlcv(exchange, max_retries, symbol, timeframe, since, limit):
    num_retries = 0
    try:
        num_retries += 1
        ohlcv = exchange.fetch_ohlcv(symbol, timeframe, since, limit)
        # print('Fetched', len(ohlcv), symbol, 'candles from', exchange.iso8601 (ohlcv[0][0]), 'to', exchange.iso8601 (ohlcv[-1][0]))
        return ohlcv
    except Exception:
        if num_retries > max_retries:
            raise  # Exception('Failed to fetch', timeframe, symbol, 'OHLCV in', max_retries, 'attempts')

In [10]:
binance = ccxt.binance()
all_ohlcv = scrape_ohlcv(binance, 3, 'XRP/USDT', '1m', '2017-01-01 00:00:00Z', 1000)

100000 XRP/USDT candles in total from 2021-09-15T01:54:00.000Z to 2021-11-23T12:33:00.000Z
200000 XRP/USDT candles in total from 2021-07-07T15:14:00.000Z to 2021-11-23T12:33:00.000Z
300000 XRP/USDT candles in total from 2021-04-29T04:34:00.000Z to 2021-11-23T12:33:00.000Z
400000 XRP/USDT candles in total from 2021-02-18T18:30:00.000Z to 2021-11-23T12:33:00.000Z
500000 XRP/USDT candles in total from 2020-12-11T08:00:00.000Z to 2021-11-23T12:33:00.000Z
600000 XRP/USDT candles in total from 2020-10-02T21:20:00.000Z to 2021-11-23T12:33:00.000Z
700000 XRP/USDT candles in total from 2020-07-25T10:40:00.000Z to 2021-11-23T12:33:00.000Z
800000 XRP/USDT candles in total from 2020-05-17T00:00:00.000Z to 2021-11-23T12:33:00.000Z
900000 XRP/USDT candles in total from 2020-03-08T13:20:00.000Z to 2021-11-23T12:33:00.000Z
1000000 XRP/USDT candles in total from 2019-12-30T08:10:00.000Z to 2021-11-23T12:33:00.000Z
1100000 XRP/USDT candles in total from 2019-10-21T23:00:00.000Z to 2021-11-23T12:33:00.00

In [13]:
dat = pd.DataFrame(all_ohlcv, columns=['time','open','high','low','close','volume'])
dat['time'] = [datetime.fromtimestamp(t/1000).strftime('%Y-%m-%d %H:%M:%S') for t in dat['time']]
dat.to_csv('XRP-USDT.csv', index=False)